먼지 데이터를 불러옵니다.

데이크루 3주차에 올리고 싶었는데 에러가 떠서 4주차에 올려봅니당 !! :)

In [2]:
# 데이터 불러오기
import pandas as pd

train = pd.read_csv("./titanic/train.csv")
test = pd.read_csv("./titanic/test.csv")
submission = pd.read_csv("./titanic/submission.csv")

In [1]:
# shape 확인하기
print(train.shape)
print(test.shape)

NameError: name 'train' is not defined

In [ ]:
train.columns

train과 test의 column은 다음과 같습니다.
- PassengerId : 탑승객의 고유 아이디
- Survival : 생존여부(0: 사망, 1: 생존)
- Pclass : 등실의 등급(1: 1등급, 2: 2등급, 3: 3등급)
- Name : 이름
- Sex : 성별
- Age : 나이
- Sibsp : 함께 탑승한 형제자매, 아내 남편의 수
- Parch: 함께 탑승한 부모, 자식의 수
- Ticket: 티켓번호
- Fare: 티켓의 요금
- Cabin: 객실번호
- Embarked: 배에 탑승한 위치(C = Cherbourg, Q = Queenstown, S = Southampton)


이 중 필요한 column만 이용하여 결과를 구해보고자 합니다.

In [ ]:
train.info()

In [ ]:
test.info()

- columns 수가 많으니 이 중 몇 개를 추려보고 싶어서 상관 관계 분석을 먼저 해보기로 합니다.

- 상관분석(Correlation Analysis)은 두 변수간에 어떤 선형적인 관계를 가지고 있는지를 분석하는 방법입니다. 이때 두 변수간의 관계의 강도를 상관관계(Correlation coefficient)라고 합니다.

- 상관계수는 두 변수간의 연관된 정도만 나타낼 뿐 원인 결과를 나타내는 인과관계를 설명하는 것은 아님을 반드시 알아야합니다.


- 상관관계 분석에서는 기본적 방법으로 피어슨 상관계수를 많이 이용합니다.

In [ ]:
train.corr()

In [ ]:
train['Sex'] = train['Sex'].replace('male',0)
test['Sex'] = test['Sex'].replace('male',0)

In [ ]:
train['Sex'] = train['Sex'].replace('female',1)
test['Sex'] = test['Sex'].replace('female',1)

In [ ]:
train.head(3)

In [ ]:
train.isnull().sum()

- Age는 없는 컬럼을 삭제해줍니다.
- Cabin은 독립 변수로 안 쓸 거라서 별다른 전처리를 하지 않을 것입니다.
- Embarked는 S,C,Q를 범주형으로 변환하려고 합니다
- 이때 Enbarked열에 존재하는 2가지 null값을 매꾸어주기 위해 null값을 제거한 후 상관관계 분석을 통해 기준 열을 이용하여 매꾸어주겠습니다.

In [ ]:
train2 = train.dropna(subset=['Embarked'])

In [ ]:
print(train2.shape)

In [ ]:
train2.head(1)

In [ ]:
import copy
train2 = train2.copy()
train2.loc[train2['Embarked'] == 'C','Embarked']=0
train2.loc[train2['Embarked'] == 'Q','Embarked']=1
train2.loc[train2['Embarked'] == 'S','Embarked']=2

In [ ]:
train2.info()

In [ ]:
train2['Embarked'] = train2['Embarked'].astype(int)

In [ ]:
# 특정 컬럼과 나머지 컬럼간의 상관계수 계산
train2.corrwith(train2['Embarked'])

- 당연한 결과겠지만 Fare와 가장 연관성이 뛰어납니다. 따라서 Fare를 살펴보고 Embarked를 채워보겠습니다.

In [ ]:
train[train['Embarked'].isnull()]

In [ ]:
train[train['Fare'] == 80.0]

In [ ]:
train[train['Pclass']==1]['Embarked'].value_counts()

In [ ]:
train[train['Survived']==1]['Embarked'].value_counts()

In [ ]:
train['Embarked'].value_counts()

- 엌ㅋㅋㅋ s가 확실이 많으니 대세를 따라서 S로 채우기로 했습니다.

In [ ]:
train['Embarked'] = train['Embarked'].fillna("S")

In [ ]:
train.isnull().sum() # 굳

In [ ]:
train.loc[train.Embarked == 'C','Embarked']=0
train.loc[train.Embarked == 'Q','Embarked']=1
train.loc[train.Embarked == 'S','Embarked']=2

In [ ]:
train['Embarked'] = train['Embarked'].astype('int64')

In [ ]:
train.corr()

- 테스트 값도 null값을 매꾸어주겠습니다,

In [ ]:
test.isnull().sum()

In [ ]:
test[test['Fare'].isnull()]

- Fare열과 Pcalss열의 상관관계가 높으므로 Pclass가 3인 행들의 Fare열의 분포를 살펴보겠습니다.

In [ ]:
fare = pd.DataFrame(test[test['Pclass']==3]['Fare'].value_counts().nlargest(12))
fare = fare.reset_index()
fare

- nlargest()<br>
가장 많은 비중을 차지하고 있는 장르를 추출하겠다는 의미입니다.

In [ ]:
# 오잉 index의 평균이어서 그렇습니다.
fare['index'].mean()

In [ ]:
test['Fare'] = test['Fare'].fillna("7.753825")

In [ ]:
test.isnull().sum()

- 조건에 맞은 Fare 값들이 특출난 값이 없기 때문에 평균값을 채워 넣어줍니다.

#### 정규화와 표준화

정규화와 표준화를 잘 보고 다르게 선택해서 하면 좋을 것 같습니다.

- 정규화 : -1 ~ 1 사이의 값으로 변환
- 표준화 : 평균이 0이고 표준편차가 1인 값으로 변환

정규화와 표준화 식은 다음과 같이 정의할 수 있습니다.

In [ ]:
def normalize(x):
  return (x-x.min())/(x.max()-x.min())

In [ ]:
def standardscaler(x):
  return (x-x.mean())/x.std()

- 정규화와 표준화 같은 경우 패키지로도 잘 나와있기 때문에 위처럼 정의를 해도 되고, 패키지를 써도 됩니다. 따라서 패키지를 이용하여 바꾸어주겠습니다.

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train[['Fare']])
train_fare = ss.transform(train[['Fare']])
test_fare = ss.transform(test[['Fare']])

In [ ]:
train['Fare2'] = train_fare
test['Fare2'] = test_fare

In [ ]:
train.head(3)

#### 이렇게 하여 완성된 열 중에서 필요한 열만 독립 변수로 가져옵니다.

In [ ]:
data = train[['Pclass','Sex','SibSp','Parch','Embarked','Fare2','Survived']]

- 여기서부터 이제 모델링을 하시면 됩니다.<br>
독립 변수 : 'Pclass','Sex','SibSp','Parch','Embarked','Fare2'<br>
종속 변수 : 'Survived'<br>

- 이 후 모델링 비교를 하시면 됩니다 :)

### 번외) Age를 독립변수를 쓰지 않은 이유


=> 편차가 너무 크고 null값이 너무 많았습니다.

In [ ]:
train['Age'].value_counts(ascending=True)

In [ ]:
train['Age'].min()

In [ ]:
train['Age'].max()

- 나이를 구간별로 바꾸려고 min과 max를 확인해보니 0살도 있고 80살도 있습니다.
- 아래는 나이 값들을 시각화해본 모습입니다.

In [ ]:
CountStatus = pd.value_counts(train['Age'].values, sort=True)
CountStatus.plot.bar()
CountStatus.plot.bar(grid=True, figsize=(10,8), fontsize=15)  # figsize, fontsize 조정

In [ ]:
train['Age'].values[:10]

In [ ]:
import matplotlib.pyplot as plt
CountStatus = pd.value_counts(train['Age'].values[:10], sort=True)
CountStatus.plot.bar()
CountStatus.plot.bar(grid=True, figsize=(10,8), fontsize=10)
plt.xticks(rotation=45)

In [ ]:
train['Age'].isnull().sum()

### [Reference]

피어슨 상관계수
https://gomguard.tistory.com/173

정규화와 표준화
https://bskyvision.com/849

In [ ]:
train_x = train[['Pclass', 'SibSp','Age', 'Sex']]
train_y = train['Survived']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, train_size=0.75, test_size=0.25)

from tpot import TPOTClassifier
pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,
                                    random_state=42, verbosity=2)
pipeline_optimizer.fit(train_x, train_y)

print(pipeline_optimizer.score(X_test, y_test))


In [ ]:
pipeline_optimizer.export('tpot_exported_pipeline.py')